In [19]:
import thingspeak
import pandas as pd
import json
import ssl
import plotly.express as px
import numpy as np
import plotly.io as pio

ssl._create_default_https_context = ssl._create_unverified_context



In [2]:
API_KEY = "ZRDS32VNQEEFSOF4"
CHANNEL_ID = "1361623"

fields = {
    "field2": "Temperature", 
    "field3":"Relative humidity", 
    "field4":"PM1", 
    "field5":"PM2.5", 
    "field6":"PM10",
}

new_fields = {v : k for (k,v) in zip(fields.keys(), fields.values())}

In [20]:
def setup_plotly():
    pd.options.plotting.backend = 'plotly'
    pio.templates.default = "plotly_dark"

setup_plotly()


In [4]:
def get_thingspeak_data(id: str, api_key: str, nb_of_results: int = 8000, drop_entry_id: bool = True, save: bool = False) -> pd.DataFrame:
    url = f"https://thingspeak.com/channels/{id}/feed.csv?apikey={api_key}&results={nb_of_results}"
    
    df = pd.read_csv(url)
    df.replace(['None'], np.nan, inplace=True)
    df.set_index("created_at", inplace=True)
    
    if drop_entry_id:
        df.drop('entry_id', axis=1, inplace=True)

    
    df.rename(columns=fields, inplace=True)

    if save:
        df.to_csv('df.csv')

    return df

In [11]:
def get_zue_data():
    df = pd.read_csv('ZUE.csv', delimiter=";")
    df.set_index("Date/time", inplace=True)

    return df


In [15]:
def extract_time_series(df: pd.DataFrame):
    pm10_series = df.loc[:, "PM10 [ug/m3]"].apply(lambda x: float(x) if not pd.isnull(x) else x)
    pm25_series = df.loc[:, "PM2.5 [ug/m3]"].apply(lambda x: float(x) if not pd.isnull(x) else x)
    temperature_series = df.loc[:, "TEMP [C]"].apply(lambda x: float(x) if not pd.isnull(x) else x)
    humidity_series = df.loc[:, "PREC [mm]"].apply(lambda x: float(x) if not pd.isnull(x) else x)

    return temperature_series, humidity_series, pm25_series, pm10_series

In [16]:
df = get_zue_data()

In [17]:
temperature_series, humidity_series, pm25_series, pm10_series = extract_time_series(df)

In [21]:
temperature_series.plot()

In [22]:
humidity_series.plot()

In [23]:
pm25_series.plot()

In [27]:
pm10_series.plot()

In [ ]:
if __name__ == "__main__":
    main()

def main():
    df = get_data(id=CHANNEL_ID, api_key=API_KEY)
    temperature_series, humidity_series, pm1_series, pm25_series, pm10_series = extract_time_series(df)
    temperature_series.plot()